# UTA
### Wykonali:
- Dominik Matracki
- Gabriel Maj
- Patryk Łyczko

In [2]:
import numpy as np
import pandas as pd 


def get_Min_Max(data):
    x, y = data.shape
    min = np.min(data,axis=0)
    max = np.max(data,axis=0)

    return min,max

def split(min,max,partitions,max_or_min,func_utility = None):
    comp_list = []
    len_data = len(partitions)
    for i in range(len_data):
        row = [(0,0)]*(partitions[i]+1)
        comp_list.append(row)

    func_utility_max = 1/len_data

    for i in range(len_data):
        if max_or_min[i] == 0:  
            comp_list[i][0] = (min[i],func_utility[i][0])
            comp_list[i][-1] = (max[i],func_utility[i][-1])
            diff = max[i] - min[i]  
            compartment = diff/partitions[i] 
            for j in range(1,partitions[i]):
                comp_list[i][j] = (min[i]+ j*compartment,func_utility[i][j])
        

        if max_or_min[i] == 1:
            comp_list[i][0] = (max[i],func_utility[i][0]) 
            comp_list[i][-1] = (min[i],func_utility[i][-1])
            diff = max[i] - min[i]
            compartment = diff/partitions[i]
            for j in range(1,partitions[i]):
                comp_list[i][j] = (max[i] - j*compartment,func_utility[i][j])

    return comp_list

def function_value(comp_elem,max_or_min):
    comp_list = []
    len_comp = len(comp_elem) - 1
    for i in range(1,len(comp_elem)):
        if max_or_min == 1:
            a = (comp_elem[(len_comp-i+1)][1]-comp_elem[(len_comp-i)][1])/(comp_elem[(len_comp-i+1)][0]-comp_elem[(len_comp-i)][0])
            b = comp_elem[(len_comp-i+1)][1]-a*comp_elem[(len_comp-i+1)][0]
            comp_list.append([a,b])
        
        if max_or_min == 0:
            a = (comp_elem[i-1][1]-comp_elem[i][1])/(comp_elem[i-1][0]-comp_elem[i][0])
            b = comp_elem[(i-1)][1]-a*comp_elem[(i-1)][0]
            comp_list.append([a,b])
    
    return comp_list

def rank(c_utility, comp_elem, point):
    score = 0  
    for i in range(len(comp_elem[0])):
        for j in range(len(comp_elem[i])-1):
            a = point[i]
            x = comp_elem[i][j][0]
            y = comp_elem[i][j+1][0]
            if point[i] <= comp_elem[i][j][0] and point[i] >= comp_elem[i][j+1][0]:
                score += c_utility[i][j][0]*point[i]+c_utility[i][j][1]

            elif point[i] >= comp_elem[i][j][0] and point[i] <= comp_elem[i][j+1][0]:
                score += c_utility[i][j][0]*point[i]+c_utility[i][j][1]

    return score

In [3]:
data_matrix = np.array([[3.6,3.7,3.9,4.5,3.5,4.5,4.6,3.7,3.1,3.6,3.7,4.2,5.1,4.7,4.2,4.6],
                        [4,12,13,6,11,4,10,9.5,17,13,15,17,20,10,14,14],
                        [4399,8199,8299,7299,5399,4549,7699,2699,4049,3200,5699,7399,10499,5099,5099,5099]])
df = pd.DataFrame(data=data_matrix, index=["CPU,taktowanie","czas pracy baterii", "Cena"])
df = df.T
df

,"CPU,taktowanie",czas pracy baterii,Cena
0,3.6,4.0,4399.0
1,3.7,12.0,8199.0
2,3.9,13.0,8299.0
3,4.5,6.0,7299.0
4,3.5,11.0,5399.0
5,4.5,4.0,4549.0
6,4.6,10.0,7699.0
7,3.7,9.5,2699.0
8,3.1,17.0,4049.0
9,3.6,13.0,3200.0


In [4]:
min_val,max_val = get_Min_Max(data_matrix)
# print(min_val, max_val)
max_min = [1,1,1]
utility = np.array([[0.34,0.18,0],[20,12,4],[0.33,0.28,0.14]])
comp = split(min_val,max_val,max_min,max_min,utility)
df_utility = pd.DataFrame(data=comp, index=["CPU,taktowanie","czas pracy baterii","cena"])
df_utility = df_utility.T
df_utility

,"CPU,taktowanie",czas pracy baterii,cena
0,"(4399.0, 0.34)","(8199.0, 20.0)","(8299.0, 0.33)"
1,"(3.6, 0.0)","(3.7, 4.0)","(3.9, 0.14)"


In [5]:
u = []
for i in max_min:
    u.append(function_value(comp[i],max_min[i]))

df_f_val = pd.DataFrame(data=u, index=["CPU,taktowanie","czas pracy baterii","cena"])
df_f_val = df_f_val.T
df_f_val

,"CPU,taktowanie",czas pracy baterii,cena
0,"[0.0019523385355020562, 3.9927763474186424]","[0.0019523385355020562, 3.9927763474186424]","[0.0019523385355020562, 3.9927763474186424]"


In [6]:
score = []
for i in data_matrix:
    score.append(rank(u,comp,i))
df_score = pd.DataFrame(data=score)
df_score

,0
0,7.999805
1,8.016790
2,32.581114
